# 租房数据分析

## 概述

本项目对各城市租房的数据进行分析，主要包括以下三个部分：

- 数据获取
- 数据分析
- 数据可视化

数据获取部分爬取了北京、上海、广州、深圳、南昌这 5 个城市全部租房数据，包含月租金、户型、朝向、面积、板块等信息。

数据分析部分分析了各城市总体租房情况租金的均价、最高价、最低价、中位数等信息，对比了各城市人均 GDP 、平均工资等信息与租房情况的关系。

数据可视化部分对以上分析进行了数量、比例、分布关系等方面的图形化直观展示。

## 数据获取



数据来源于链家官网的租房数据。

首先导入爬取数据需要的库：

In [16]:
import requests # 用于获取网页内容
from bs4 import BeautifulSoup as bs # 用于解析网页内容
import time # 用于延时，避免爬取过快被封IP
import csv # 用于将数据写入 csv 文件

为方便开发时对代码进行调试，设置调试模式开关。调试时打开，最后正确代码执行时关闭，避免输出过多影响报告内容。

伪造请求头用于简单的反爬虫

In [17]:
# 调试模式开关
debug = True

# 伪造请求头
headers = {
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) \
                   AppleWebKit/537.36 (KHTML, like Gecko) \
                   Chrome/119.0.0.0 Safari/537.36'
}

为提高代码的复用性和可扩展性，以下是爬虫程序所用到的函数，最后只需要调用 `scrape()` 即可执行完整的爬虫程序：

In [18]:
def getHTMLText(url: str) -> str:
    """获取单个网页的 HTML 字符串内容

    Args:
        url (str): 网页的 URL

    Returns:
        str: 网页的 HTML 字符串内容
    """
    try:
        r = requests.get(url, timeout=30, headers=headers)
        r.raise_for_status() # 如果状态不是200，引发HTTPError异常
        r.encoding = r.apparent_encoding # 使得解码正确
        return r.text
    except:
        return None

def getRentList(rentURL: str) -> list:
    """获取单个网页的租房信息列表

    Args:
        rentURL (str): 网页的 URL

    Raises:
        Exception: 租房信息格式不匹配

    Returns:
        list: 某单个网页租房信息列表，包括标题、行政区、板块、小区、面积、朝向、户型、租金
    """
    rentList = []

    # 获取网页内容
    html = getHTMLText(rentURL)
    if html == None:
        print(f'{rentURL}访问异常') if debug == True else None
        return rentList
    
    # 解析网页内容
    soup = bs(html, 'html.parser')
    house_list = soup.find_all('div', attrs={'class': 'content__list--item--main'})

    for house in house_list:
        # 租房信息的标题
        p = house.find('p', attrs={'class': 'content__list--item--title'})
        if p == None:
            p = house.find('p', attrs={'class': 'content__list--item--title twoline'})
        title = p.find('a').text.strip()
        # 租房信息的描述
        description = house.find('p', attrs={'class': 'content__list--item--des'})
        # 位置
        location = description.find_all('a', attrs={'target': '_blank'})
        # location可能没有
        if len(location) == 0:
            print(f'[{title}] 缺少位置信息，已跳过...') if debug == True else None
            continue
        # 行政区
        district = location[0].text.strip() # district
        # 板块
        block = location[1].text.strip() # block
        # 小区
        community = location[2].text.strip() # community
        try:
            # 把面积、朝向、户型、租金分开
            des = description.get_text(strip=True).split('/')
            # 有时候有广告，第一个元素是 "精选"，手动去除
            if des[0] == '精选':
                des.pop(0)
            # 面积如果是范围取平均值
            area_range = des[1].replace('㎡', '').strip().split('-')
            area = float(area_range[0]) if len(area_range) == 1\
                                        else (float(area_range[0]) + float(area_range[1])) / 2
            # 朝向
            direction = des[2].replace(' ', '')
            check = ['东', '南', '西', '北']
            if not any([ch in direction for ch in check]):
                print(f'[{title}] 朝向不是东南西北中的一个') if debug == True else None
                raise Exception # 朝向不是东南西中的一个，格式不匹配
            # 户型
            type = des[3].strip()
        except:
            print(f'[{title}] 租房信息格式不匹配，已跳过...') if debug == True else None
            continue
        # 租金如果是范围取平均值
        price_range = house.find('span', attrs={'class': 'content__list--item-price'})\
                    .find('em').text.strip()\
                    .split('-')
        price = float(price_range[0]) if len(price_range) == 1\
                                    else (float(price_range[0]) + float(price_range[1])) / 2
        # 将信息添加到列表中
        rentList.append([title, district, block, community, area, direction, type, price])

    time.sleep(1) # 延时 1s
    return rentList

def getCityRent(city: str, page: int) -> list:
    """获取一个城市若干页的租房信息列表

    Args:
        city (str): 城市的缩写，与链家网站的 URL 一致
        page (int): 要爬取的页数

    Returns:
        list: 某城市若干页的租房信息列表，包括标题、行政区、板块、小区、面积、朝向、户型、租金
    """
    rentList = []
    for i in range(1, page + 1):
        print(f'正在爬取第 {i} 页...') if debug == True else None
        rentURL = f'https://{city}.lianjia.com/zufang/pg{i}/'
        rentList += getRentList(rentURL)
        time.sleep(1) # 延时 1s
    return rentList

def saveRentList(rentList: list, city: str):
    """将租房信息列表保存到 csv 文件中

    Args:
        rentList (list): 租房信息列表
        city (str): 城市的缩写，与链家网站的 URL 一致
    """
    print(f'正在保存 {city} 租房信息...')
    with open(f'{city}_rent.csv', 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['标题', '行政区', '板块', '小区', '面积', '朝向', '户型', '租金'])
        writer.writerows(rentList)
    print(f'{city} 租房信息保存成功!')

def scrape():
    """爬取租房信息，爬虫程序入口
    """
    cites = ['bj', 'sh', 'gz', 'sz', 'nc'] # 爬取的城市
    page = 10 # 爬取的页数
    for city in cites:
        print(f'正在爬取 {city} 租房信息...') if debug == True else None
        rentList = getCityRent(city, page)
        saveRentList(rentList, city)
        print(f'{city} 爬取完成!')

执行爬虫进行数据获取：

In [ ]:
scrape() # 启动爬虫程序

所爬取的五个城市信息存储在以下五个 csv 文件中：

- 北京：`bj_rent.csv`
- 上海：`sh_rent.csv`
- 广州：`gz_rent.csv`
- 深圳：`sz_rent.csv`
- 南昌：`nc_rent.csv`

表头格式为：

```
标题,行政区,板块,小区,面积,朝向,户型,租金
```

## 数据分析

## 数据可视化